In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoModelForSequenceClassification,AutoTokenizer, BertTokenizer
from datasets import Dataset,DatasetDict

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [32]:
df_train_essay = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
df_train_essay.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [3]:
# df_train_prompt = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")
# df_train_prompt.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [4]:
# df_combined = df_train_essay.merge(df_train_prompt, how = 'left', on = 'prompt_id')
# df_combined.head()

,id,prompt_id,text,generated,prompt_name,instructions,source_text
0,0059830c,0,Cars. Cars have been around since they became ...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,005db917,0,Transportation is a large necessity in most co...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
3,00940276,0,How often do you ride in a car? Do you drive a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."


In [5]:
# df_combined['combined_column'] = 'Prompt: ' + df_combined['prompt_name'] + ': Instructions:' + df_combined['instructions'] + ': Source text:' + df_combined['source_text'] + ': Outcome:' + df_combined['text']
# df_combined.head()

,id,prompt_id,text,generated,prompt_name,instructions,source_text,combined_column
0,0059830c,0,Cars. Cars have been around since they became ...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",Prompt: Car-free cities: Instructions:Write an...
1,005db917,0,Transportation is a large necessity in most co...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",Prompt: Car-free cities: Instructions:Write an...
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",Prompt: Car-free cities: Instructions:Write an...
3,00940276,0,How often do you ride in a car? Do you drive a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",Prompt: Car-free cities: Instructions:Write an...
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",Prompt: Car-free cities: Instructions:Write an...


In [35]:
model = 'microsoft/deberta-v3-small'
# model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/")
df_filter = df_train_essay[['generated','text']].rename(columns={"text":"input", "generated":"labels"})
# df_filter['text'].astype(str)
df_filter['labels'] = df_filter['labels'].astype(float)
ds = Dataset.from_pandas(df_filter)
ds[0]
# df_filter.columns

{'labels': 0.0,
 'input': 'Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, people are starting to question if limiting car usage would be a good thing. To me, limiting the use of cars might be a good thing to do.\n\nIn like matter of this, article, "In German Suburb, Life Goes On Without Cars," by Elizabeth Rosenthal states, how automobiles are the linchpin of suburbs, where middle class families from either Shanghai or Chicago tend to make their homes. Experts say how this is a huge impediment to current efforts to reduce greenhouse gas emissions from tailpipe. Passenger cars are responsible for 12 percent of greenhouse gas emissions in Europe...and up to 50 percent in some carintensive areas in the United States. Cars are the main reason for the greenhouse gas emissions because of a lot of people driving them around all the time getting wher

In [8]:
from transformers import DebertaV2Tokenizer

In [36]:
# tokz = DebertaV2Tokenizer.from_pretrained(model)
tokz = AutoTokenizer.from_pretrained(model)
def tok_func(examples):
    result = tokz(
        examples["input"],
        truncation=True,
        max_length=512,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result
# def tok_func(x): return tokz(str(x["input"]), truncation=True)

tok_ds = ds.map(tok_func, batched=True)


/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?ba/s]

In [37]:
from transformers import TrainingArguments,Trainer
# tok_ds = tokz(ds)
bs =16
epochs =4
lr = 8e-5

dds = tok_ds.train_test_split(0.25, seed=42)
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

ml = AutoModelForSequenceClassification.from_pretrained(model, num_labels=1)
trainer = Trainer(ml, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz)

trainer.train()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.003195
2,No log,0.003786
3,No log,0.003196
4,No log,0.003190


TrainOutput(global_step=236, training_loss=0.0035805108688645444, metrics={'train_runtime': 332.7374, 'train_samples_per_second': 22.612, 'train_steps_per_second': 0.709, 'total_flos': 996702435717120.0, 'train_loss': 0.0035805108688645444, 'epoch': 4.0})

In [3]:
# ml.save_pretrained("/kaggle/working/pretrained_bert/")
trainer.save_model("/kaggle/working/finetuned_bert/")

NameError: name 'trainer' is not defined

In [30]:
# model1 = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/")

In [29]:
df_test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
df_test

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [23]:
# df_test['combined_column'] = 'Prompt: ' + df_test['prompt_name'] + ': Instructions:' + df_test['instructions'] + ': Source text:' + df_test['source_text'] + ': Outcome:' + df_test['text']
df_test_updt = df_test[['text']].rename(columns={"text":"input"})
eval_ds = Dataset.from_pandas(df_test_updt).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[0.00214808],
       [0.00241538],
       [0.00129989]])

In [26]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': df_test['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

75

In [23]:
# dds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [16]:
# row = tok_ds[0]
# row

{'labels': 0.0,
 'input': 'Prompt: Car-free cities: Instructions:Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.: Source text:# In German Suburb, Life Goes On Without Cars by Elisabeth Rosenthal\n\n1 VAUBAN, Germany—Residents of this upscale community are suburban pioneers, going where few soccer moms or commuting executives have ever gone before: they have given up their cars.\n\n2 Street parking, driveways and home garages are generally forbidden in this experimental new district on the outskirts of Freiburg, near the French an

In [ ]:
# from torch import nn

# pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
# print(pt_predictions)

In [ ]:
# pt_batch